In [1]:
import msprime
import pyslim
import tskit

# Generating initial diversity

## Annotating a tree sequence with SLiM metadata

Here we run an msprime simulation:

In [2]:
demog_model = msprime.Demography()
demog_model.add_population(initial_size=1_000)
ts = msprime.sim_ancestry(
        samples=1_000,
        demography=demog_model,
        random_seed=5,
        recombination_rate=1e-8,
        sequence_length = 90_000_000)

and then "annotate" it so it can be loaded into SLiM.

In [3]:
ats = pyslim.annotate(ts, model_type="WF", tick=1, stage="late")

That inserted default metadata in many objects; to see what those defaults are, we can do as follows, for instance:

In [4]:
pyslim.default_slim_metadata("individual")

{'pedigree_id': -1,
 'age': -1,
 'subpopulation': -1,
 'sex': -1,
 'flags': 0,
 'pedigree_p1': -1,
 'pedigree_p2': -1}

## Adding mutations with SLiM metadata

Now, we can add some SLiM mutations to the coalescent simulation
(*note:* the seed is set to be a special one that gets stacked mutations; see below):

In [5]:
mut_model = msprime.SLiMMutationModel(type=2)
mts = msprime.sim_mutations(ats, rate=3e-10, model=mut_model, random_seed=573)
print(f"New mutations added: {mts.num_mutations}")

New mutations added: 921


But, we'd like them to be non-neutral, so we edit them:

In [6]:
tables = mts.dump_tables()
tables.mutations.clear()
for m in mts.mutations():
  md_list = m.metadata["mutation_list"]
  for md in md_list:
    md["selection_coeff"] = 0.001
    tables.mutations.append(
        m.replace(metadata={"mutation_list": md_list})
    )
sts = tables.tree_sequence()

We'd like to demonstrate what happens if more than one mutation
hits the same site, so let's find one:

In [7]:
# Finding stacked mutations
for mut in sts.mutations():
    if len(mut.metadata["mutation_list"]) > 1:
        print(mut.id)

622
623


Here's what one looks like: it's a superposition of SLiM mutations 621 and 622.
(Note: selection coefficients are stored as (single-precision) floats,
but they're being printed as doubles,
which is why the values for `selection_coeff` look funny.

In [8]:
print(sts.mutation(622))

Mutation(id=622, site=621, node=5607, derived_state='621,622', parent=621, metadata={'mutation_list': [{'mutation_type': 2, 'selection_coeff': 0.0010000000474974513, 'subpopulation': -1, 'slim_time': -987, 'nucleotide': -1}, {'mutation_type': 2, 'selection_coeff': 0.0, 'subpopulation': -1, 'slim_time': -8, 'nucleotide': -1}]}, time=9.684319995964623, edge=22712)


## Loading into SLiM

First, we dump the tree sequence to file:

In [9]:
sts.dump("init.trees")

Then we can run the SLiM script, which loads that file:

In [10]:
! slim -d "L=90000000" reload_annotated.slim 

// Initial random seed:
410124043951279033

// RunInitializeCallbacks():
initializeSLiMModelType(modelType = 'WF');
initializeTreeSeq();
initializeMutationRate(3e-10);
initializeMutationType(1, 0.5, "f", 0);
initializeMutationType(2, 0.5, "e", 0.001);
initializeGenomicElementType(1, m2, 1);
initializeGenomicElement(g1, 0, 89999999);
initializeRecombinationRate(1e-08);

// Starting run at tick <start>:
1 



That script saved the output to `final.trees`, which we can now look at:

In [11]:
fts = tskit.load("final.trees")
fts